In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
file_path = "https://raw.githubusercontent.com/Premalatha-success/Datasets/refs/heads/main/loan_prediction.csv"
df = pd.read_csv(file_path)

# Fill missing values in key columns
df["Gender"] = df["Gender"].fillna("Unknown")
df["Married"] = df["Married"].fillna("No")
df["Dependents"] = df["Dependents"].fillna("0")  # Filling missing Dependents with 0
df["LoanAmount"] = df["LoanAmount"].fillna(df["LoanAmount"].mean())
df["Loan_Amount_Term"] = df["Loan_Amount_Term"].fillna(df["Loan_Amount_Term"].mean())
df["Credit_History"] = df["Credit_History"].fillna(1.0)  # Filling missing credit history with 1

# Encode categorical variables
le_gender = LabelEncoder()
df["Gender"] = le_gender.fit_transform(df["Gender"])

le_married = LabelEncoder()
df["Married"] = le_married.fit_transform(df["Married"])

# Encode "Dependents" (including "3+")
df["Dependents"] = df["Dependents"].replace("3+", "3")
df["Dependents"] = df["Dependents"].astype(int)

le_education = LabelEncoder()
df["Education"] = le_education.fit_transform(df["Education"])

le_self_employed = LabelEncoder()
df["Self_Employed"] = df["Self_Employed"].fillna("No")
df["Self_Employed"] = le_self_employed.fit_transform(df["Self_Employed"])

le_property_area = LabelEncoder()
df["Property_Area"] = le_property_area.fit_transform(df["Property_Area"])

le_loan_status = LabelEncoder()
df["Loan_Status"] = le_loan_status.fit_transform(df["Loan_Status"])

# Define features and target
X = df.drop(["Loan_ID", "Loan_Status"], axis=1)
y = df["Loan_Status"]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train the model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluation
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Fix unknown values in prediction function
def predict_loan(Gender, Married, Dependents, Education, Self_Employed, ApplicantIncome,
                 CoapplicantIncome, LoanAmount, Loan_Amount_Term, Credit_History, Property_Area):
    try:
        # Handle unknown values
        if Gender not in le_gender.classes_:
            return "Invalid Gender Input"
        if Married not in le_married.classes_:
            return "Invalid Married Input"
        if Self_Employed not in le_self_employed.classes_:
            return "Invalid Self-Employed Input"
        if Property_Area not in le_property_area.classes_:
            return "Invalid Property Area Input"

        # Encode categorical inputs
        Gender = le_gender.transform([Gender])[0]
        Married = le_married.transform([Married])[0]
        Education = le_education.transform([Education])[0]
        Self_Employed = le_self_employed.transform([Self_Employed])[0]
        Property_Area = le_property_area.transform([Property_Area])[0]

        # Convert "3+" to 3 and ensure Dependents is handled properly
        if Dependents == "3+":
            Dependents = 3
        Dependents = int(Dependents)

        # Prepare input array
        input_data = np.array([[Gender, Married, Dependents, Education, Self_Employed, ApplicantIncome,
                                CoapplicantIncome, LoanAmount, Loan_Amount_Term, Credit_History,
                                Property_Area]])

        # Scale input
        input_data = scaler.transform(input_data)

        # Predict
        prediction = model.predict(input_data)[0]
        return "Loan Approved" if prediction == 1 else "Loan Rejected"

    except Exception as e:
        return f"Error: {str(e)}"

# Deploy with Gradio
inputs = [
    gr.Dropdown(le_gender.classes_.tolist(), label="Gender"),
    gr.Dropdown(le_married.classes_.tolist(), label="Married"),
    gr.Dropdown(["0", "1", "2", "3+"], label="Dependents"),  # Include "3+"
    gr.Dropdown(le_education.classes_.tolist(), label="Education"),
    gr.Dropdown(le_self_employed.classes_.tolist(), label="Self Employed"),
    gr.Number(label="Applicant Income"),
    gr.Number(label="Coapplicant Income"),
    gr.Number(label="Loan Amount"),
    gr.Number(label="Loan Amount Term"),
    gr.Radio([0.0, 1.0], label="Credit History"),  # 0 for no history, 1 for history
    gr.Dropdown(le_property_area.classes_.tolist(), label="Property Area")
]

output = gr.Textbox(label="Loan Prediction")

gr.Interface(fn=predict_loan, inputs=inputs, outputs=output, title="Loan Approval Prediction Model").launch(share=True, debug=True)


Model Accuracy: 0.78
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://aceadfb10e368d3780.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
